In [ ]:
import pandas as pd
import json
import datetime
import time
import os
import numpy as np
import lightgbm as lgb
from config import DATA_ENTHALPY_GRADIENTS_PROCESSED_FILE, DATA_BIM_RAW_FOLDER
import matplotlib
from enthalpygradients import EnthalpyGradient
from daily_enthalpy_gradient_module import daily_enthalpy_gradients_hourly_data

In [ ]:
gradients_output_data_path = DATA_ENTHALPY_GRADIENTS_PROCESSED_FILE

In [ ]:
weather_data = pd.read_csv(os.path.join(DATA_BIM_RAW_FOLDER, "weather2018.csv"))
weather_data = weather_data.append(pd.read_csv(os.path.join(DATA_BIM_RAW_FOLDER, "weather2019.csv")), ignore_index=True)

In [ ]:
#get weather data
weather_data['timestamp'] = pd.to_datetime(weather_data['timestamp'])
weather_data = weather_data.set_index('timestamp', drop=False)
weather_data

In [ ]:
# local variables
Temperature_base_C = 26
Relative_humidity_base_perc = 70

# Initialize class
eg = EnthalpyGradient(Temperature_base_C, Relative_humidity_base_perc)

# calculate enthalpy gradients for certain outdoor conditions for one year (8760 hours)
Temperature_outdoor_C = weather_data['Temperature_C'].values
Relative_humidity_outdoor_perc = weather_data['RH_%'].values

## total specific thermal energy consumption
how = 'hourly'
type = 'dehumidification'
q_dehum_kWh_m2 = eg.specific_thermal_consumption(Temperature_outdoor_C, Relative_humidity_outdoor_perc, how=how, type=type)
type = 'cooling'
q_cooling_kWh_m2 = eg.specific_thermal_consumption(Temperature_outdoor_C, Relative_humidity_outdoor_perc, how=how, type=type)

In [ ]:
weather_data['q_total_kWh_m2'] = q_dehum_kWh_m2 + q_cooling_kWh_m2

In [ ]:
result = weather_data.resample('D').sum()
result['consumption'] = result['q_total_kWh_m2'] * 100 #typical area
ax = result[['consumption']].plot(legend=False)
font = {'family' : 'Arial',
        'size'   : 18}
ax.set_xlabel("")
ax.set_ylabel("Daily Consumption [kWh]")
matplotlib.rc('font', **font)

In [ ]:
#this gates saved
daily_enthalpy_gradients['consumption'] = daily_enthalpy_gradients_hourly_data(weather_data.index, weather_data['Temperature_C'].values, weather_data['RH_%'].values)
daily_enthalpy_gradients
daily_enthalpy_gradients['timestamp'] = daily_enthalpy_gradients.index
daily_enthalpy_gradients[['timestamp', 'DEG_C_kJperKg', 'DEG_DEHUM_kJperKg']].to_csv(gradients_output_data_path, index=False)

In [ ]:
len(q_total_kWh_m2)

In [ ]:
daily_enthalpy_gradients